In [16]:
import numpy as np
import random
import plotly.express as px

In [ ]:
cities = [
    "Chabarovice",
    "Usti nad Labem",
    "Praha",
    "Brno",
    "Karlovy Vary",
    "Kladno",
    "Ml. Boleslav",
    "Vsetaty",
]

adj_matrix = np.array(
    [
        [0, 10, 92, 296, 111, 123, 179, 87],
        [10, 0, 91, 295, 119, 206, 55, 159],
        [92, 91, 0, 205, 129, 93, 217, 76],
        [296, 295, 205, 0, 336, 185, 224, 54],
        [111, 119, 129, 336, 0, 137, 202, 98],
        [123, 206, 93, 185, 137, 0, 208, 71],
        [179, 55, 217, 224, 202, 208, 0, 126],
        [87, 159, 76, 54, 98, 71, 126, 0],
    ]
)



def distance(citya, cityb, cities, adj_matrix):
    return adj_matrix[cities.index(citya), cities.index(cityb)]


def fitness(paths, cities, adj_matrix):
    return [
        sum([distance(path[i], path[(i + 1) % len(path)], cities, adj_matrix) for i in range(len(path))])
        for path in paths
    ]


def select(fit_values, n_parents, paths):
    cumsum = np.cumsum(fit_values / sum(fit_values))
    selected = []
    while len(selected) < n_parents:
        r = random.random()
        for i in cumsum:
            if i > r:
                selected.append(np.where(cumsum == i)[0][0])
                break
    return selected

def crossover(n_paths, selected, paths):
    next_generation = []
    while len(next_generation) < n_paths:
        parents = random.sample(tuple(selected), 2)  # Convert selected to a tuple
        parent1, parent2 = paths[parents[0]], paths[parents[1]]
        child = [None] * len(parent1)

        # Inherit a random subset from parent1
        subset_size = random.randint(0, len(parent1) - 1)
        subset_start = random.randint(0, len(parent1) - subset_size)
        subset_end = subset_start + subset_size
        child[subset_start:subset_end] = parent1[subset_start:subset_end]

        # Fill in the remaining cities from parent2
        parent2_index = 0
        for i in range(len(child)):
            if child[i] is None:
                while parent2[parent2_index] in child:
                    parent2_index += 1
                child[i] = parent2[parent2_index]
                parent2_index += 1

        next_generation.append(child)

    return next_generation

def tsp_gen(n_paths, n_parents, cities, adj_matrix, n_gens):
    paths = np.array([np.random.permutation(cities) for i in range(n_paths)])
    min = (0, 9999)

    for gen in range(n_gens):
        selected = select(fitness(paths, cities, adj_matrix), n_parents, paths)
        paths = crossover(n_paths, selected, paths)
        print(gen, np.min(fitness(paths, cities, adj_matrix)))
        if np.min(fitness(paths, cities, adj_matrix)) < min[1]:
            min = (gen, np.min(fitness(paths, cities, adj_matrix)))
            
    return min


tsp_gen(20, 10, cities, adj_matrix, 1000)


In [28]:
import numpy as np
import random

def distance(citya, cityb, cities, adj_matrix):
    return adj_matrix[cities.index(citya), cities.index(cityb)]

def fitness(paths, cities, adj_matrix):
    return [
        sum([distance(path[i], path[(i + 1) % len(path)], cities, adj_matrix) for i in range(len(path))])
        for path in paths
    ]

def select(fit_values, n_parents):
    cumsum = np.cumsum(fit_values / sum(fit_values))
    selected = []
    while len(selected) < n_parents:
        r = random.random()
        for i in cumsum:
            if i > r:
                selected.append(np.where(cumsum == i)[0][0])
                break
    return selected


def crossover(n_paths, selected, paths):
    """
    4 je polovina 1 rodice

    Args:
        n_paths (_type_): _description_
        selected (_type_): _description_
        paths (_type_): _description_

    Returns:
        _type_: _description_
    """
    next_gen = []
    while len(next_gen) < n_paths:
        parents = random.sample(selected, 2)
        parent1, parent2 = paths[parents[0]], paths[parents[1]]
        child = [None for i in range(len(parent1))]

        subset_start = random.randint(0, len(parent1) - 4)
        child[subset_start:subset_start + 4] = parent1[subset_start:subset_start + 4]

        parent2_index = 0
        for i in range(len(child)):
            while child[i] is None:
                if parent2[parent2_index] in child:
                    parent2_index += 1
                else:
                    child[i] = parent2[parent2_index]
                    parent2_index += 1

        # parent2_index = 0
        # for i in range(len(child)):
            # if child[i] is None:
                # while parent2[parent2_index] in child:
                    # parent2_index += 1
                # child[i] = parent2[parent2_index]
                # parent2_index += 1

        next_gen.append(child)

    return next_gen


def mutate(paths, mutation_rate):
    for path in paths:
        if random.random() < mutation_rate:
            swap = random.sample(range(len(path)), 2)
            path[swap[0]], path[swap[1]] = path[swap[1]], path[swap[0]]
    return paths

def tsp_gen(n_paths, n_parents, cities, adj_matrix, n_gens, mutation_rate=0.01):
    gens = []
    paths = np.array([np.random.permutation(cities) for i in range(n_paths)])
    min_distance = 9999
    best_path = None

    for gen in range(n_gens):
        fit_values = fitness(paths, cities, adj_matrix)
        selected = select(fit_values, n_parents)
        paths = crossover(n_paths, selected, paths)
        paths = mutate(paths, mutation_rate)

        current_min = np.min(fit_values)
        if current_min < min_distance:
            min_distance = current_min
            best_path = paths[np.argmin(fit_values)]
        gens.append((gen, min_distance))
        print(f"Generace {gen}: Fitness = {min_distance}")

    return best_path, min_distance, gens


cities = [
    "Chabarovice",
    "Usti nad Labem",
    "Praha",
    "Brno",
    "Karlovy Vary",
    "Kladno",
    "Ml. Boleslav",
    "Vsetaty",
]

adj_matrix = np.array(
    [
        [0, 10, 92, 296, 111, 123, 179, 87],
        [10, 0, 91, 295, 119, 206, 55, 159],
        [92, 91, 0, 205, 129, 93, 217, 76],
        [296, 295, 205, 0, 336, 185, 224, 54],
        [111, 119, 129, 336, 0, 137, 202, 98],
        [123, 206, 93, 185, 137, 0, 208, 71],
        [179, 55, 217, 224, 202, 208, 0, 126],
        [87, 159, 76, 54, 98, 71, 126, 0],
    ]
)


best_path, min_distance, gens = tsp_gen(8, 4, cities, adj_matrix, 10000)

print("Best Path:", best_path)
print("Minimum Distance:", min_distance)

x_values = [t[0] for t in gens]
y_values = [t[1] for t in gens]


fig = px.line(x=x_values, y = y_values)
fig.show()

Generace 0: Fitness = 1037
Generace 1: Fitness = 911
Generace 2: Fitness = 911
Generace 3: Fitness = 911
Generace 4: Fitness = 911
Generace 5: Fitness = 911
Generace 6: Fitness = 911
Generace 7: Fitness = 911
Generace 8: Fitness = 911
Generace 9: Fitness = 911
Generace 10: Fitness = 911
Generace 11: Fitness = 911
Generace 12: Fitness = 911
Generace 13: Fitness = 911
Generace 14: Fitness = 911
Generace 15: Fitness = 911
Generace 16: Fitness = 911
Generace 17: Fitness = 911
Generace 18: Fitness = 911
Generace 19: Fitness = 911
Generace 20: Fitness = 911
Generace 21: Fitness = 911
Generace 22: Fitness = 911
Generace 23: Fitness = 911
Generace 24: Fitness = 911
Generace 25: Fitness = 911
Generace 26: Fitness = 911
Generace 27: Fitness = 911
Generace 28: Fitness = 911
Generace 29: Fitness = 911
Generace 30: Fitness = 911
Generace 31: Fitness = 911
Generace 32: Fitness = 911
Generace 33: Fitness = 911
Generace 34: Fitness = 911
Generace 35: Fitness = 911
Generace 36: Fitness = 911
Generace 3

In [ ]:
sez = ['Ml. Boleslav', 'Brno', 'Kladno', 'Vsetaty', 'Praha', 'Chabarovice', 'Karlovy Vary', 'Usti nad Labem']